In [ ]:
# Yield Curves, Term Structure, Interest Rate Dynamics, Introduction to QuantLib

In [8]:
import QuantLib as ql

In [9]:
# Construct yield curve
calc_date = ql.Date(1, 1, 2017)
ql.Settings.instance().evaluationDate = calc_date

spot_dates = [ql.Date(1,1,2017), ql.Date(1,1,2018), ql.Date(1,1,2027)]
spot_rates = [0.04, 0.04, 0.04]

In [10]:
day_count = ql.SimpleDayCounter()
calendar = ql.NullCalendar()
interpolation = ql.Linear()
compounding = ql.Compounded
compounding_frequency = ql.Annual
spot_curve = ql.ZeroCurve(spot_dates, spot_rates, day_count, calendar, interpolation, compounding, compounding_frequency)

spot_curve_handle = ql.YieldTermStructureHandle(spot_curve)

In [12]:
# Construct bond schedule
issue_date = ql.Date(1, 1, 2017)
maturity_date = ql.Date(1, 1, 2022)
tenor = ql.Period(ql.Semiannual)
calendar = ql.NullCalendar()
business_convention = ql.Unadjusted
date_generation = ql.DateGeneration.Backward
month_end = False

schedule = ql.Schedule(issue_date, maturity_date, tenor, calendar, business_convention, business_convention, date_generation, month_end)

In [13]:
# Create FixedRateBond Object
coupon_rate = 0.05
coupons = [coupon_rate]
settlement_days = 0
face_value = 100

fixed_rate_bond = ql.FixedRateBond(settlement_days,
                                face_value,
                                schedule,
                                coupons,
                                day_count)

In [14]:
for c in fixed_rate_bond.cashflows():
    print('%20s %12f' % (c.date(), c.amount()))

      July 1st, 2017     2.500000
   January 1st, 2018     2.500000
      July 1st, 2018     2.500000
   January 1st, 2019     2.500000
      July 1st, 2019     2.500000
   January 1st, 2020     2.500000
      July 1st, 2020     2.500000
   January 1st, 2021     2.500000
      July 1st, 2021     2.500000
   January 1st, 2022     2.500000
   January 1st, 2022   100.000000


In [ ]:
# Set Valuation engine
bond_engine = DiscountingBondEngine(spot_curve_handle)
fixed_rate_bond.setPricingEngine(bond_engine)

# Calculate present value
value = fixed_rate_bond.NPV()
print(value)

In [1]:
def print_curve(xlist, ylist, precision=3):
    """
    Method to print curve in a nice format
    """
    print("----------------------")
    print("Maturities\tCurve")
    print("----------------------")
    for x,y in zip(xlist, ylist):
        print(x,"\t\t", round(y, precision))
    print("----------------------")

In [2]:
# Deposit rates
depo_maturities = [ql.Period(6,ql.Months), ql.Period(12, ql.Months)]
depo_rates = [5.25, 5.5]

# Bond rates
bond_maturities = [ql.Period(6*i, ql.Months) for i in range(3,21)]
bond_rates = [5.75, 6.0, 6.25, 6.5, 6.75, 6.80, 7.00, 7.1, 7.15,
              7.2, 7.3, 7.35, 7.4, 7.5, 7.6, 7.6, 7.7, 7.8]

print_curve(depo_maturities+bond_maturities, depo_rates+bond_rates)

----------------------
Maturities	Curve
----------------------
6M 		 5.25
1Y 		 5.5
1Y6M 		 5.75
2Y 		 6.0
2Y6M 		 6.25
3Y 		 6.5
3Y6M 		 6.75
4Y 		 6.8
4Y6M 		 7.0
5Y 		 7.1
5Y6M 		 7.15
6Y 		 7.2
6Y6M 		 7.3
7Y 		 7.35
7Y6M 		 7.4
8Y 		 7.5
8Y6M 		 7.6
9Y 		 7.6
9Y6M 		 7.7
10Y 		 7.8
----------------------


In [3]:
# some constants and conventions
# here we just assume for the sake of example
# that some of the constants are the same for
# depo rates and bond rates

calc_date = ql.Date(15, 1, 2015)
ql.Settings.instance().evaluationDate = calc_date

calendar = ql.UnitedStates()
bussiness_convention = ql.Unadjusted
day_count = ql.Thirty360()
end_of_month = True
settlement_days = 0
face_amount = 100
coupon_frequency = ql.Period(ql.Semiannual)
settlement_days = 0

In [4]:
# create deposit rate helpers from depo_rates
depo_helpers = [ql.DepositRateHelper(ql.QuoteHandle(ql.SimpleQuote(r/100.0)),
                                     m,
                                     settlement_days,
                                     calendar,
                                     bussiness_convention,
                                     end_of_month,
                                     day_count )
                for r, m in zip(depo_rates, depo_maturities)]

In [5]:
# create fixed rate bond helpers from fixed rate bonds
bond_helpers = []
for r, m in zip(bond_rates, bond_maturities):
    termination_date = calc_date + m
    schedule = ql.Schedule(calc_date,
                   termination_date,
                   coupon_frequency,
                   calendar,
                   bussiness_convention,
                   bussiness_convention,
                   ql.DateGeneration.Backward,
                   end_of_month)

    helper = ql.FixedRateBondHelper(ql.QuoteHandle(ql.SimpleQuote(face_amount)),
                                        settlement_days,
                                        face_amount,
                                        schedule,
                                        [r/100.0],
                                        day_count,
                                        bussiness_convention,
                                        )
    bond_helpers.append(helper)

In [6]:
rate_helpers = depo_helpers + bond_helpers
yieldcurve = ql.PiecewiseLogCubicDiscount(calc_date,
                             rate_helpers,
                             day_count)

In [7]:
# get spot rates
spots = []
tenors = []
for d in yieldcurve.dates():
    yrs = day_count.yearFraction(calc_date, d)
    compounding = ql.Compounded
    freq = ql.Semiannual
    zero_rate = yieldcurve.zeroRate(yrs, compounding, freq)
    tenors.append(yrs)
    eq_rate = zero_rate.equivalentRate(day_count,
                                       compounding,
                                       freq,
                                       calc_date,
                                       d).rate()
    spots.append(100*eq_rate)